In [1]:
from data_reader import DataReader
import datasets
from tweet_to_vec import TweetToVec
import utils

In [2]:
dr = DataReader('nlkt')

dr.read_dataset(datasets.binary_classes)
dr.read_dataset(datasets.ternary_classes)



100%|██████████████████████████████████| 1000/1000 [00:00<00:00, 1753471.57it/s]


In [3]:
embeddings = dr.read_embeddings('embeddings/kraby.txt')
method = 'fixed_length'
L = 8

t2v = TweetToVec(embeddings, method, L)

100%|██████████████████████████████████| 20067/20067 [00:00<00:00, 59367.93it/s]


In [4]:
batch_size = 32

binary_dataset = dr.get_dataset('binary')
ternary_dataset = dr.get_dataset('ternary')

vectorized_binary = t2v.vectorize_dataset(binary_dataset)
vectorized_ternary = t2v.vectorize_dataset(ternary_dataset)

batched_binary = t2v.batch_dataset(vectorized_binary, batch_size)
batched_ternary = t2v.batch_dataset(vectorized_ternary, batch_size)

In [5]:
from simple_network import SimpleNetwork

In [6]:
import torch
import torch.nn as nn
import torch.optim as optim

simple_model = nn.Sequential(
                nn.Linear(800, 2))

ce_loss = torch.nn.functional.cross_entropy

def my_optimizer_sgd(model_parameters):
    return optim.SGD(model_parameters, lr = 0.001, momentum = 0.9)

In [7]:
network = SimpleNetwork(simple_model, ce_loss)
network.train(batched_binary['training tweets'], batched_binary['training tags'], 50, my_optimizer_sgd)

After epoch 0 tot_loss = 100.42084503173828
After epoch 1 tot_loss = 80.03201293945312
After epoch 2 tot_loss = 75.88964080810547
After epoch 3 tot_loss = 73.803955078125
After epoch 4 tot_loss = 72.49606323242188
After epoch 5 tot_loss = 70.96530151367188
After epoch 6 tot_loss = 70.20743560791016
After epoch 7 tot_loss = 69.67229461669922
After epoch 8 tot_loss = 69.2419204711914
After epoch 9 tot_loss = 68.88249969482422
After epoch 10 tot_loss = 68.06693267822266
After epoch 11 tot_loss = 67.69448852539062
After epoch 12 tot_loss = 67.44424438476562
After epoch 13 tot_loss = 67.23172760009766
After epoch 14 tot_loss = 67.0438003540039
After epoch 15 tot_loss = 66.44403839111328
After epoch 16 tot_loss = 66.18143463134766
After epoch 17 tot_loss = 66.02074432373047
After epoch 18 tot_loss = 65.8832778930664
After epoch 19 tot_loss = 65.75975036621094
After epoch 20 tot_loss = 65.2766342163086
After epoch 21 tot_loss = 65.06971740722656
After epoch 22 tot_loss = 64.95272827148438
Aft

In [8]:
binary_predictions = network.predict(batched_binary['test tweets'])

In [9]:
utils.save_results(binary_predictions, 'results/binary_simple_network.txt')

In [10]:
!perl graders/evaluate1.pl results/binary_simple_network.txt

Precision = 41.58%
Recall = 31.34%
Balanced F-score = 35.74%
Accuracy = 84.90%
